In [1]:
import pandas as pd
import numpy as np

# 可視化用のライブラリ
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import neologdn
import MeCab

import re

from tqdm import tqdm_notebook as tqdm
from tqdm._tqdm_notebook import tqdm_notebook
# # プログレスバーのラベル設定
tqdm_notebook.pandas(desc="progress: ")

In [2]:
df = pd.read_pickle('posi_nega_alldata.pkl')
df.head()

,text,posi_nega
0,案件個別の技術相談乗ってます！乗ります,1
1,勉強とお仕事が有機的に繋がっていて面白い。,1
2,次々とテクニックを教えていただき、ためになりました,1
3,お肉がキレイに焼けたことに小さなしあわせを感じた,1
4,これが私の最適化,1


# 形態素解析結果をarray形式で出力、形態素解析結果をデータフレーム型としてリストに格納、そのリストを縦に連結したデータフレームとして出力

In [3]:
'''形態素解析結果をarray形式で出力する関数'''
def get_wakati_text_array(text):
    #np.nanがstr型ではなくfloat型とされるためstr型以外はneologdn.normalizeメソッドを実施しないように条件分岐
        if not isinstance(text, str):
            pass
        else:            
            text_normalization = neologdn.normalize(text)
            # 大文字英字を小文字英字に変更
            text_normalization = text_normalization.lower()    
            neologd_tagger = MeCab.Tagger('-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
            wakati_text = neologd_tagger.parse(text_normalization).replace('\n', '\t').split("\t")[:-2]
            wakati_results_array = np.array([wakati_text]).reshape(-1, 6)
            
            return wakati_results_array



'''array形式で出力された形態素解析結果をデータフレーム型としてリストに格納、そのリストを縦に連結したデータフレームとして出力する関数'''
def create_wakati_df(wakati_array):
    
    wakati_list = [pd.DataFrame(i) for i in wakati_array]
    wakati_df = pd.concat(wakati_list, axis=0)\
    .rename(columns={0:'surface', 1:'yomi', 2:'original', 3:'type', 4:'katsuyoukei', 5:'katsuyougata'})
    
    return wakati_df

### 上記関数を実行

In [4]:
# 今回apply関数をSeries型（text_df['text']）に対して使用するためaxis=1を引数に与えない
wakati_array = df['text'].progress_apply(get_wakati_text_array)

wakati_df = create_wakati_df(wakati_array)

wakati_df.head()

,surface,yomi,original,type,katsuyoukei,katsuyougata
0,案件,アンケン,案件,名詞-一般,,
1,個別,コベツ,個別,名詞-形容動詞語幹,,
2,の,ノ,の,助詞-連体化,,
3,技術,ギジュツ,技術,名詞-一般,,
4,相談,ソウダン,相談,名詞-サ変接続,,
